In [ ]:
import os

def listar_directorio(raiz='.', profundidad=3):
    for root, dirs, files in os.walk(raiz):
        nivel = root.replace(raiz, '').count(os.sep)
        if nivel >= profundidad:
            continue
        sangría = '│   ' * nivel
        print(f"{sangría}📁 {os.path.basename(root)}/")
        for f in files:
            print(f"{sangría}│   📄 {f}")

listar_directorio('.')


In [ ]:
from pathlib import Path

def obtener_archivos_vacios(raiz='.'):
    archivos_vacios = []
    for archivo in Path(raiz).rglob('*'):
        if archivo.is_file() and archivo.stat().st_size == 0:
            archivos_vacios.append(archivo)
    return archivos_vacios

archivos = obtener_archivos_vacios()
print(f"📦 Archivos vacíos detectados: {len(archivos)}\n")
for archivo in archivos:
    print(f"📝 {archivo}")


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Cargar .env desde raíz del proyecto
load_dotenv()
client = OpenAI()

# Función para generar contenido con un prompt
def generar_contenido(nombre_archivo):
    prompt = f"""
Quiero que escribas el contenido para un archivo llamado '{nombre_archivo.name}' como parte de un proyecto de agente general en Python.

El archivo debe cumplir con su propósito según su nombre, y debe tener una estructura profesional, modular y clara. 

Usa buenas prácticas como:
- comentarios explicativos
- separación de funciones si aplica
- si es __init__.py, prepáralo para que pueda importar módulos del paquete
- si es un script principal, incluye punto de entrada `if __name__ == "__main__"`

No uses explicaciones, solo dame el código final del archivo.
"""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


In [ ]:
from pathlib import Path

# Ruta raíz del proyecto
ruta_raiz = Path().resolve()

# Buscar archivos .py vacíos en subdirectorios
archivos_para_generar = []
for archivo in ruta_raiz.rglob("*.py"):
    if archivo.name == "generate_files.py":
        continue  # evitamos regenerar este mismo script
    if archivo.stat().st_size == 0:
        archivos_para_generar.append(archivo)

# Mostrar lista
print("Archivos vacíos detectados para generar contenido:\n")
for a in archivos_para_generar:
    print("🟦", a.relative_to(ruta_raiz))


In [ ]:
import os
from dotenv import load_dotenv

# Cargar las variables del entorno desde el archivo .env
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("❌ La clave OPENAI_API_KEY no está definida en el archivo .env. Por favor, revisa.")
else:
    print("✅ Clave API de OpenAI cargada correctamente.")


In [ ]:
from pathlib import Path

def escanear_directorio(base_dir=".", nivel_max=3, prefijo=""):
    base = Path(base_dir).resolve()
    resultado = []

    def recorrer(ruta, nivel):
        if nivel > nivel_max:
            return
        for item in sorted(ruta.iterdir()):
            linea = f"{'    ' * nivel}- {item.name}"
            resultado.append(linea)
            if item.is_dir():
                recorrer(item, nivel + 1)

    recorrer(base, 0)
    return "\n".join(resultado)

estructura = escanear_directorio(".", 3)
print(estructura)


In [ ]:
from openai import OpenAI

client = OpenAI()  # Usa la API key del entorno automáticamente

# Prompt general para generar un README.md basado en la estructura del proyecto
prompt = f"""
Eres un experto en desarrollo de software. A continuación tienes la estructura de un proyecto.

{estructura}

Con base en esta estructura, genera un archivo README.md profesional. Incluye:

- Nombre del proyecto
- Descripción del propósito general
- Instrucciones para instalar dependencias
- Cómo ejecutar el proyecto
- Ejemplo de uso
- Créditos (puedes dejar como plantilla si no se especifican)

Inclúyelo como si fuera el contenido completo de README.md.
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

contenido_readme = response.choices[0].message.content
print(contenido_readme)


In [ ]:
prompt = f"""
Basado en la siguiente estructura de proyecto:

{estructura}

Y considerando que el proyecto incluye un agente en Python que se conecta a la API de OpenAI, carga variables con dotenv, y puede usarse desde terminal o integrarse con Nautilus, genera el contenido de un archivo requirements.txt mínimo pero funcional para que el sistema funcione correctamente. Usa versiones estables.
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2
)

requirements = response.choices[0].message.content
print(requirements)


In [ ]:
import os
from pathlib import Path

# Ruta base del proyecto
ruta_base = Path().resolve()

# Archivos detectados
archivos_vacios = []

for ruta in ruta_base.rglob("*"):
    if ruta.is_file() and ruta.stat().st_size == 0:
        archivos_vacios.append(ruta)

print("📁 Archivos vacíos encontrados:")
for archivo in archivos_vacios:
    print(" -", archivo.relative_to(ruta_base))


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Cargar clave de API
load_dotenv()
client = OpenAI()

# Configura aquí la ruta base de tu proyecto
BASE_DIR = "."

# Extensiones válidas
EXT_VALIDAS = {".py", ".sh", ".md", ".txt"}

# Archivos que no queremos tocar
EXCLUIR = {
    "requirements.txt",
    ".env",
    ".env.example",
    "README.md",
    "pyproject.toml",
    "Dockerfile",
}

def es_valido(archivo):
    nombre = os.path.basename(archivo)
    ext = os.path.splitext(nombre)[1]
    if nombre in EXCLUIR:
        return False
    if ext not in EXT_VALIDAS:
        return False
    if os.path.getsize(archivo) > 10:  # más de 10 bytes = ya tiene contenido
        return False
    return True

def sugerencia_para(ruta):
    prompt = f"Actúa como programador experto. Dado el archivo '{ruta}', escribe el contenido inicial adecuado para ese archivo dentro de un proyecto Python moderno. No incluyas explicaciones, solo el código o texto."
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()

# Recorremos y completamos
for root, _, files in os.walk(BASE_DIR):
    for file in files:
        ruta = os.path.join(root, file)
        if es_valido(ruta):
            print(f"✍️ Generando contenido para: {ruta}")
            contenido = sugerencia_para(ruta)
            with open(ruta, "w", encoding="utf-8") as f:
                f.write(contenido)
print("✅ Archivos completados.")
